# Slow Motion

In [1]:
# import libraries
import cv2
import numpy as np

In [7]:
video = cv2.VideoCapture('videos/handball1.mp4')
if not video.isOpened(): 
    print("Unable to read video file")

# Getting some infomations about movie
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)
n_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
video_time = n_frames / fps

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('results/handball.avi',fourcc, fps, (int(width),int(height)))

print("Width:", width)
print("Height:", height)
print("FPS:", fps)
print("Nº frames:", n_frames)
print("Video time:", video_time)

Width: 1280
Height: 720
FPS: 30.0
Nº frames: 709.0
Video time: 23.633333333333333


### Processing frames

In [8]:
def check_pos(width, height, a, b):
    return (a >= width) or (a < 0) or (b >= height) or (b < 0)

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = width * height,
                       qualityLevel = 0.1,
                       minDistance = 5,
                       blockSize = 7 )
                       
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (50,50),
                  maxLevel = 5,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))


# Take first frame and find corners in it
_, old_frame = video.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

while video.isOpened():
    _ , frame = video.read()
    if frame is None:
        break
        
    new_frame1 = frame.copy()
    new_frame2 = frame.copy()
    new_frame3 = frame.copy()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

    # calculate optical flow
    p1, _, _ = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    
    # draw the tracks
    for i,(new,old) in enumerate(zip(p1,p0)):
        a,b = old.ravel()
        c,d = new.ravel()
        
        if check_pos(width, height, a, b) or check_pos(width, height, c, d):
            continue
        
        w_2 = (a + c) // 2 - 1
        h_2 = (b + d) // 2 - 1
        w_1 = 0
        h_1 = 0
        w_3 = 0
        h_3 = 0
        
        if a < c:
            w_1 = (a + w_2) // 2 - 1
            w_3 = (w_2 + c) // 2 - 1
        else:
            w_3 = (a + w_2) // 2 - 1
            w_1 = (w_2 + c) // 2 - 1
            
        if b < d:
            h_1 = (b + h_2) // 2 - 1
            h_3 = (h_2 + d) // 2 - 1
        else:
            h_3 = (b + h_2) // 2 - 1
            h_1 = (h_2 + d) // 2 - 1
            
        new_frame1[int(h_1),int(w_1)] = old_frame[int(b),int(a)]
        new_frame2[int(h_2),int(w_2)] = old_frame[int(b),int(a)]
        new_frame3[int(h_3),int(w_3)] = old_frame[int(b),int(a)]
    
    out.write(old_frame)
    out.write(new_frame1)
    out.write(new_frame2)
    out.write(new_frame3)
            
    # Now update the previous frame and previous points
    old_frame = frame.copy()
    old_gray = frame_gray.copy()
    
video.release()
out.release()